In [424]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/advertising-cmaes/advertising.csv


In [425]:
!pip install deap

In [426]:
import random
import operator
import csv
import itertools
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [427]:
import pandas as pd 
df = pd.read_csv("../input/advertising-cmaes/advertising.csv",header=None)
df.drop([4,5,7,8], axis = 1, inplace=True)
df.columns = [0,1,2,3,4,5]
print(df.head(5))


                          0    1            2                     3     4  \
0  Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  Male   
1                     68.95   35      61833.9                256.09     0   
2                     80.23   31     68441.85                193.77     1   
3                     69.47   26     59785.94                 236.5     0   
4                     74.15   29     54806.18                245.89     1   

               5  
0  Clicked on Ad  
1              0  
2              0  
3              0  
4              0  


In [428]:
df = df.drop(0)
print(df.head(5))

       0   1         2       3  4  5
1  68.95  35   61833.9  256.09  0  0
2  80.23  31  68441.85  193.77  1  0
3  69.47  26  59785.94   236.5  0  0
4  74.15  29  54806.18  245.89  1  0
5  68.37  35  73889.99  225.58  0  0


In [429]:
 
print(df.head(5))

       0   1         2       3  4  5
1  68.95  35   61833.9  256.09  0  0
2  80.23  31  68441.85  193.77  1  0
3  69.47  26  59785.94   236.5  0  0
4  74.15  29  54806.18  245.89  1  0
5  68.37  35  73889.99  225.58  0  0


In [430]:
print(df.columns)
df[5] = df[5].astype('int').astype('bool')
print(df.head(10))

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')
        0   1         2       3  4      5
1   68.95  35   61833.9  256.09  0  False
2   80.23  31  68441.85  193.77  1  False
3   69.47  26  59785.94   236.5  0  False
4   74.15  29  54806.18  245.89  1  False
5   68.37  35  73889.99  225.58  0  False
6   59.99  23  59761.56  226.74  1  False
7   88.91  33  53852.85  208.36  0  False
8    66.0  48  24593.33  131.76  1   True
9   74.53  30   68862.0  221.51  1  False
10  69.88  20  55642.32  183.82  1  False


In [431]:
train, test = train_test_split(df, test_size=0.2)
print(train.shape)
print(train.head(10))
train_data = train.values.tolist()
print(test.head(10))

(800, 6)
         0   1         2       3  4      5
599  83.53  36  67686.16  204.56  0  False
703  87.27  30  51824.01  204.27  1   True
697  39.19  54  52581.16  173.05  0   True
119  84.45  50  29727.79  207.18  0   True
533  79.53  51  46132.18  244.91  0  False
362  62.14  41  59397.89  110.93  1   True
326  36.87  36  29398.61  195.91  0   True
830  48.46  49  61230.03  132.38  1   True
885  70.92  39  66522.79  249.81  1  False
883  86.81  29  50820.74  199.62  1  False
         0   1         2       3  4      5
57   65.19  36  75254.88  150.61  0   True
121  86.19  31  57669.41  210.26  1  False
98   39.94  41  64927.19   156.3  0   True
792  35.49  47  36884.23  170.04  0   True
411  48.09  33  19345.36  180.42  0   True
621  81.75  24  52656.13  190.08  1  False
839  44.13  40  33601.84  128.48  1   True
291  50.52  31  72270.88  171.62  0   True
573   72.8  35  63551.67  249.54  0  False
950  78.96  50  69868.48  193.15  0   True


In [432]:
train_data_new = list(list(float(elem) if elem != '?' else float(0) for elem in row) for row in train_data)
print(train_data_new)

[[83.53, 36.0, 67686.16, 204.56, 0.0, 0.0], [87.27, 30.0, 51824.01, 204.27, 1.0, 1.0], [39.19, 54.0, 52581.16, 173.05, 0.0, 1.0], [84.45, 50.0, 29727.79, 207.18, 0.0, 1.0], [79.53, 51.0, 46132.18, 244.91, 0.0, 0.0], [62.14, 41.0, 59397.89, 110.93, 1.0, 1.0], [36.87, 36.0, 29398.61, 195.91, 0.0, 1.0], [48.46, 49.0, 61230.03, 132.38, 1.0, 1.0], [70.92, 39.0, 66522.79, 249.81, 1.0, 0.0], [86.81, 29.0, 50820.74, 199.62, 1.0, 0.0], [41.84, 49.0, 37605.11, 139.32, 0.0, 1.0], [80.23, 31.0, 68441.85, 193.77, 1.0, 0.0], [80.94, 36.0, 60803.0, 239.94, 0.0, 0.0], [74.53, 30.0, 68862.0, 221.51, 1.0, 0.0], [50.08, 30.0, 41629.86, 123.91, 0.0, 1.0], [75.84, 21.0, 48537.18, 186.98, 0.0, 0.0], [80.67, 34.0, 58909.36, 239.76, 0.0, 0.0], [72.07, 31.0, 69710.51, 226.45, 0.0, 0.0], [76.79, 27.0, 55677.12, 235.94, 0.0, 0.0], [46.37, 52.0, 32847.53, 144.27, 0.0, 1.0], [42.39, 42.0, 66541.05, 150.99, 0.0, 1.0], [45.48, 49.0, 53336.76, 129.16, 1.0, 1.0], [79.81, 37.0, 51847.26, 253.17, 0.0, 0.0], [66.14, 41.0

In [433]:
print(train)

         0   1         2       3  4      5
599  83.53  36  67686.16  204.56  0  False
703  87.27  30  51824.01  204.27  1   True
697  39.19  54  52581.16  173.05  0   True
119  84.45  50  29727.79  207.18  0   True
533  79.53  51  46132.18  244.91  0  False
..     ...  ..       ...     ... ..    ...
627  85.77  27  52261.73  191.78  1  False
452  55.92  24  33951.63  145.08  0   True
432  88.89  35  50439.49   218.8  1  False
689  74.58  36  70701.31  230.52  0  False
808  53.22  44  44307.18  108.85  0   True

[800 rows x 6 columns]


In [434]:
train_res = train[5]
test_res = test[5]
print(train_res)

599    False
703     True
697     True
119     True
533    False
       ...  
627    False
452     True
432    False
689    False
808     True
Name: 5, Length: 800, dtype: bool


In [435]:
print(test_res)

57      True
121    False
98      True
792     True
411     True
       ...  
245    False
95      True
211    False
56     False
585     True
Name: 5, Length: 200, dtype: bool


In [436]:
pset = gp.PrimitiveSetTyped("MAIN", itertools.repeat(float, 4), bool, "IN")

pset.addPrimitive(operator.and_, [bool, bool], bool)
pset.addPrimitive(operator.or_, [bool, bool], bool)
pset.addPrimitive(operator.not_, [bool], bool)

In [437]:
def protectedDiv(left, right):
    try: return left / right
    except ZeroDivisionError: return 1

In [438]:
pset.addPrimitive(operator.add, [float,float], float)
pset.addPrimitive(operator.sub, [float,float], float)
pset.addPrimitive(operator.mul, [float,float], float)
pset.addPrimitive(protectedDiv, [float,float], float)

In [439]:

def if_then_else(input, output1, output2):
    if input: return output1
    else: return output2

In [440]:
pset.addPrimitive(operator.lt, [float, float], bool)
pset.addPrimitive(operator.eq, [float, float], bool)
pset.addPrimitive(if_then_else, [bool, float, float], float)
pset.addTerminal(False, bool)
pset.addTerminal(True, bool)

In [441]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [442]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [443]:
def evalTrainbase(individual):
    func = toolbox.compile(expr=individual)
    train_data_2_sample = random.sample(train_data_new, 200)
    result = sum(bool(func(*val[:4])) is bool(val[4]) for val in train_data_2_sample)
    return result,

In [444]:
toolbox.register("evaluate", evalTrainbase)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [445]:
random.seed(10)
pop = toolbox.population(n=100)

In [446]:

hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)
    
algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 50, stats, halloffame=hof)

print(pop, stats, hof)

gen	nevals	avg	std    	min	max
0  	100   	100	5.88897	86 	113
1  	62    	102.82	6.09488	86 	115
2  	60    	104.2 	5.8    	89 	115
3  	51    	106.03	6.1424 	91 	115
4  	69    	104.52	6.89852	85 	120
5  	63    	103.9 	7.16589	87 	115
6  	60    	105.31	6.54323	89 	115
7  	70    	105.33	7.36757	83 	118
8  	68    	105.54	7.37485	91 	120
9  	62    	105.36	8.04676	87 	120
10 	63    	104.46	8.56086	87 	120
11 	57    	105.19	7.71971	87 	120
12 	63    	105.05	7.4691 	87 	120
13 	70    	104.7 	8.64234	87 	121
14 	61    	105.57	7.68928	89 	121
15 	66    	105.19	7.60749	83 	121
16 	61    	106.13	7.66636	87 	121
17 	59    	106.01	8.22252	85 	121
18 	51    	106.86	8.97889	83 	121
19 	68    	105.62	7.95711	90 	121
20 	63    	105.1 	6.88404	91 	121
21 	62    	104.21	6.54568	87 	118
22 	56    	105.19	7.16198	86 	118
23 	65    	103.93	7.87179	87 	118
24 	58    	105.4 	6.72161	91 	118
25 	58    	105.76	7.40422	85 	118
26 	44    	107.5 	7.86702	83 	119
27 	58    	106.98	8.05727	87 	119
28 	67    	106.56	8.

In [447]:
hof

In [448]:
pop[0]

In [449]:
print(test.head(5))

         0   1         2       3  4      5
57   65.19  36  75254.88  150.61  0   True
121  86.19  31  57669.41  210.26  1  False
98   39.94  41  64927.19   156.3  0   True
792  35.49  47  36884.23  170.04  0   True
411  48.09  33  19345.36  180.42  0   True


In [450]:
test_data = test.values.tolist()
print(test_data)

[['65.19', '36', '75254.88', '150.61', '0', True], ['86.19', '31', '57669.41', '210.26', '1', False], ['39.94', '41', '64927.19', '156.3', '0', True], ['35.49', '47', '36884.23', '170.04', '0', True], ['48.09', '33', '19345.36', '180.42', '0', True], ['81.75', '24', '52656.13', '190.08', '1', False], ['44.13', '40', '33601.84', '128.48', '1', True], ['50.52', '31', '72270.88', '171.62', '0', True], ['72.8', '35', '63551.67', '249.54', '0', False], ['78.96', '50', '69868.48', '193.15', '0', True], ['39.56', '41', '59243.46', '143.13', '1', True], ['75.03', '34', '35764.49', '255.57', '0', True], ['70.92', '41', '49995.63', '108.16', '1', True], ['39.86', '36', '32593.59', '145.85', '0', True], ['88.82', '36', '58638.75', '169.1', '0', False], ['68.11', '38', '73884.48', '231.21', '0', False], ['87.46', '37', '61009.1', '211.56', '1', False], ['61.09', '26', '55336.18', '131.68', '1', True], ['57.7', '34', '42696.67', '109.07', '0', True], ['68.94', '54', '30726.26', '138.71', '0', True]

In [451]:
test_data_new = list(list(float(elem) if elem != '?' else float(0) for elem in row) for row in test_data)
print(test_data_new)

[[65.19, 36.0, 75254.88, 150.61, 0.0, 1.0], [86.19, 31.0, 57669.41, 210.26, 1.0, 0.0], [39.94, 41.0, 64927.19, 156.3, 0.0, 1.0], [35.49, 47.0, 36884.23, 170.04, 0.0, 1.0], [48.09, 33.0, 19345.36, 180.42, 0.0, 1.0], [81.75, 24.0, 52656.13, 190.08, 1.0, 0.0], [44.13, 40.0, 33601.84, 128.48, 1.0, 1.0], [50.52, 31.0, 72270.88, 171.62, 0.0, 1.0], [72.8, 35.0, 63551.67, 249.54, 0.0, 0.0], [78.96, 50.0, 69868.48, 193.15, 0.0, 1.0], [39.56, 41.0, 59243.46, 143.13, 1.0, 1.0], [75.03, 34.0, 35764.49, 255.57, 0.0, 1.0], [70.92, 41.0, 49995.63, 108.16, 1.0, 1.0], [39.86, 36.0, 32593.59, 145.85, 0.0, 1.0], [88.82, 36.0, 58638.75, 169.1, 0.0, 0.0], [68.11, 38.0, 73884.48, 231.21, 0.0, 0.0], [87.46, 37.0, 61009.1, 211.56, 1.0, 0.0], [61.09, 26.0, 55336.18, 131.68, 1.0, 1.0], [57.7, 34.0, 42696.67, 109.07, 0.0, 1.0], [68.94, 54.0, 30726.26, 138.71, 0.0, 1.0], [81.51, 41.0, 53412.32, 250.03, 0.0, 0.0], [74.32, 33.0, 62060.11, 128.17, 0.0, 1.0], [70.9, 28.0, 71727.51, 190.95, 0.0, 0.0], [55.13, 45.0, 55

In [452]:
def evalTest(individual):
    func = toolbox.compile(expr=individual)
    result = sum(1 if (func(*val[:4]) == bool(val[4])) else 0 for val in test_data_new)
    return result

In [453]:
for ind in pop:
    print(evalTest(ind))

99
112
112
112
112
112
112
112
112
112
112
88
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
88
112
112
99
88
112
112
112
112
112
112
112
112
112
112
112
99
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
99
112
112
112
112
112
112
112
112
112
112
112
112
112
88
112
88
112
112
112
112
112
112
112
88
112
112
112


In [454]:
toolbox.compile(expr=pop[0])

<function <lambda>(IN0, IN1, IN2, IN3)>

In [455]:
print(test_data_new)

[[65.19, 36.0, 75254.88, 150.61, 0.0, 1.0], [86.19, 31.0, 57669.41, 210.26, 1.0, 0.0], [39.94, 41.0, 64927.19, 156.3, 0.0, 1.0], [35.49, 47.0, 36884.23, 170.04, 0.0, 1.0], [48.09, 33.0, 19345.36, 180.42, 0.0, 1.0], [81.75, 24.0, 52656.13, 190.08, 1.0, 0.0], [44.13, 40.0, 33601.84, 128.48, 1.0, 1.0], [50.52, 31.0, 72270.88, 171.62, 0.0, 1.0], [72.8, 35.0, 63551.67, 249.54, 0.0, 0.0], [78.96, 50.0, 69868.48, 193.15, 0.0, 1.0], [39.56, 41.0, 59243.46, 143.13, 1.0, 1.0], [75.03, 34.0, 35764.49, 255.57, 0.0, 1.0], [70.92, 41.0, 49995.63, 108.16, 1.0, 1.0], [39.86, 36.0, 32593.59, 145.85, 0.0, 1.0], [88.82, 36.0, 58638.75, 169.1, 0.0, 0.0], [68.11, 38.0, 73884.48, 231.21, 0.0, 0.0], [87.46, 37.0, 61009.1, 211.56, 1.0, 0.0], [61.09, 26.0, 55336.18, 131.68, 1.0, 1.0], [57.7, 34.0, 42696.67, 109.07, 0.0, 1.0], [68.94, 54.0, 30726.26, 138.71, 0.0, 1.0], [81.51, 41.0, 53412.32, 250.03, 0.0, 0.0], [74.32, 33.0, 62060.11, 128.17, 0.0, 1.0], [70.9, 28.0, 71727.51, 190.95, 0.0, 0.0], [55.13, 45.0, 55

In [456]:
def predict(individual):
    func = toolbox.compile(expr=individual)
    result = list(func(*val[:4]) for val in test_data_new)
    return result

In [457]:
accuracy_score(test_res, predict(pop[0]))

0.62